In [1]:
import pandas as pd
import sqlite3
import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

from sqlalchemy import create_engine

In [2]:
#connection to a PostgreSQL local database
# ===========================================

# user_name_var = input('What is the username of your database? ')
host_var = input('What is the host address of your database?')
pw = input("Enter the database password: ")
# db_name = input("Enter the name of the database: ")

string_output = "postgresql://postgres"+':'+pw+'@'+host_var+":5432"+'/earthquake_db'
engine = create_engine(string_output)
connection = engine.connect()
print("Connected to database...")

Connected to database...


In [3]:
#create and connect to a sqlite3 db
conn = sqlite3.connect(r'/Users/rosaicelaroman/Desktop/portfolio_work/earthquake_modeling/Database/db/earthquake_db')

Adding 1 month of data

In [4]:
page_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"

df = pd.read_csv(page_url)
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-03-17T18:24:24.450Z,34.062667,-117.266833,15.140000,1.41,ml,42.0,70.00,0.11360,0.2100,...,2022-03-17T18:27:59.388Z,"2km NNW of Loma Linda, CA",earthquake,0.29,0.49,0.169000,30.0,automatic,ci,ci
1,2022-03-17T18:13:22.750Z,19.189167,-155.334335,44.700001,2.07,md,35.0,192.00,NaN,0.1300,...,2022-03-17T18:16:40.060Z,"15 km E of Pāhala, Hawaii",earthquake,0.88,1.10,0.880000,3.0,automatic,hv,hv
2,2022-03-17T18:05:53.539Z,63.044500,-151.012000,127.700000,1.60,ml,NaN,NaN,NaN,0.4500,...,2022-03-17T18:08:35.739Z,"62 km NNW of Petersville, Alaska",earthquake,NaN,1.20,NaN,NaN,automatic,ak,ak
3,2022-03-17T17:50:12.990Z,33.349667,-116.293833,11.150000,1.05,ml,41.0,65.00,0.01829,0.1800,...,2022-03-17T18:06:50.570Z,"13km NE of Borrego Springs, CA",earthquake,0.21,0.34,0.138000,21.0,reviewed,ci,ci
4,2022-03-17T17:35:04.970Z,46.856667,-121.927167,6.650000,0.31,ml,12.0,83.00,0.02058,0.0800,...,2022-03-17T18:18:08.700Z,"13 km NE of Ashford, Washington",earthquake,0.80,0.63,0.017013,3.0,reviewed,uw,uw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9539,2022-02-15T19:12:23.740Z,39.421500,-110.312333,-1.560000,1.83,md,9.0,194.00,0.01022,0.1200,...,2022-02-15T19:51:17.520Z,"15 km SSE of Sunnyside, Utah",earthquake,0.63,0.36,0.324000,7.0,reviewed,uu,uu
9540,2022-02-15T19:04:27.070Z,38.825000,-122.797167,2.680000,0.41,md,25.0,44.00,0.01021,0.0300,...,2022-02-17T09:24:12.854Z,"6km NNW of The Geysers, CA",earthquake,0.18,0.28,0.129000,3.0,reviewed,nc,nc
9541,2022-02-15T19:00:38.627Z,38.537500,-119.382100,0.000000,1.40,ml,6.0,179.74,0.05500,0.1995,...,2022-02-16T02:37:07.010Z,"8 km ENE of Walker, California",earthquake,NaN,0.00,0.280000,3.0,reviewed,nn,nn
9542,2022-02-15T18:52:46.384Z,55.372400,-159.286100,15.000000,1.80,ml,NaN,NaN,NaN,0.5500,...,2022-03-05T17:45:05.111Z,"60 km SSE of Ivanof Bay, Alaska",earthquake,NaN,0.40,NaN,NaN,reviewed,ak,ak


In [5]:
#append the new data from USGS to both databases
#=============
#PostgreSQL
df.to_sql('earthquake_raw', engine, if_exists='append', index = False)
#=============
#SQLite
cur = conn.cursor()
df.to_sql('earthquake_raw', conn, if_exists='append', index=False)

Load from database to update with new rows and remove duplicate rows

In [6]:
#Dataframe from postgresql
model_df = pd.read_sql_table(
    "earthquake_raw",
    con=engine
)
model_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.3840,-117.107500,1.10,1.30,ml,8.0,149.46,0.87500,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.1810,-117.818000,7.30,1.40,ml,11.0,154.64,0.11900,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.3407,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.8703,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.0145,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37874,2022-02-15T19:12:23.740Z,39.4215,-110.312333,-1.56,1.83,md,9.0,194.00,0.01022,0.1200,...,2022-02-15T19:51:17.520Z,"15 km SSE of Sunnyside, Utah",earthquake,0.63,0.36,0.324,7.0,reviewed,uu,uu
37875,2022-02-15T19:04:27.070Z,38.8250,-122.797167,2.68,0.41,md,25.0,44.00,0.01021,0.0300,...,2022-02-17T09:24:12.854Z,"6km NNW of The Geysers, CA",earthquake,0.18,0.28,0.129,3.0,reviewed,nc,nc
37876,2022-02-15T19:00:38.627Z,38.5375,-119.382100,0.00,1.40,ml,6.0,179.74,0.05500,0.1995,...,2022-02-16T02:37:07.010Z,"8 km ENE of Walker, California",earthquake,NaN,0.00,0.280,3.0,reviewed,nn,nn
37877,2022-02-15T18:52:46.384Z,55.3724,-159.286100,15.00,1.80,ml,NaN,NaN,NaN,0.5500,...,2022-03-05T17:45:05.111Z,"60 km SSE of Ivanof Bay, Alaska",earthquake,NaN,0.40,NaN,NaN,reviewed,ak,ak


In [7]:
#dataframe from SQLite
model_df2 = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
model_df2

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.3840,-117.107500,1.10,1.30,ml,8.0,149.46,0.87500,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.1810,-117.818000,7.30,1.40,ml,11.0,154.64,0.11900,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.3407,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.8703,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.0145,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33889,2022-02-15T19:12:23.740Z,39.4215,-110.312333,-1.56,1.83,md,9.0,194.00,0.01022,0.1200,...,2022-02-15T19:51:17.520Z,"15 km SSE of Sunnyside, Utah",earthquake,0.63,0.36,0.324,7.0,reviewed,uu,uu
33890,2022-02-15T19:04:27.070Z,38.8250,-122.797167,2.68,0.41,md,25.0,44.00,0.01021,0.0300,...,2022-02-17T09:24:12.854Z,"6km NNW of The Geysers, CA",earthquake,0.18,0.28,0.129,3.0,reviewed,nc,nc
33891,2022-02-15T19:00:38.627Z,38.5375,-119.382100,0.00,1.40,ml,6.0,179.74,0.05500,0.1995,...,2022-02-16T02:37:07.010Z,"8 km ENE of Walker, California",earthquake,NaN,0.00,0.280,3.0,reviewed,nn,nn
33892,2022-02-15T18:52:46.384Z,55.3724,-159.286100,15.00,1.80,ml,NaN,NaN,NaN,0.5500,...,2022-03-05T17:45:05.111Z,"60 km SSE of Ivanof Bay, Alaska",earthquake,NaN,0.40,NaN,NaN,reviewed,ak,ak


In [8]:
#check for duplicates in postgres data
boolean = model_df.duplicated(subset=['id']).any()
print(boolean)

True


In [9]:
#check for duplicates in sqlite data
boolean = model_df2.duplicated(subset=['id']).any()
print(boolean)

False


In [10]:
#drop duplicates in postgres data
model_df.drop_duplicates(subset=['id'], inplace=True)
print(model_df)

                           time  latitude  longitude  depth  mag magType  \
0      2022-01-06T01:42:39.855Z   37.3840  -117.1075    1.1  1.3      ml   
1      2022-01-06T01:31:33.698Z   38.1810  -117.8180    7.3  1.4      ml   
2      2022-01-06T01:24:20.083Z   61.3407  -149.8852   30.0  1.5      ml   
3      2022-01-06T01:15:15.851Z   62.8703  -150.7625  111.3  2.1      ml   
4      2022-01-06T01:10:44.629Z   64.0145  -148.7657    1.9  1.5      ml   
...                         ...       ...        ...    ...  ...     ...   
37864  2022-02-15T20:00:03.795Z   60.3836  -151.1242   53.5  1.3      ml   
37867  2022-02-15T19:43:07.466Z   56.1342  -153.1985    3.5  3.0      ml   
37868  2022-02-15T19:39:13.870Z   63.0288  -150.6892   98.8  1.1      ml   
37871  2022-02-15T19:25:38.691Z   58.6191  -154.4855    5.8  1.6      ml   
37877  2022-02-15T18:52:46.384Z   55.3724  -159.2861   15.0  1.8      ml   

        nst     gap   dmin     rms  ...                   updated  \
0       8.0  149.4

In [11]:
#drop duplicates in sqlite data
model_df2.drop_duplicates(subset=['id'], inplace=True)
print(model_df2)

                           time  latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z   37.3840 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z   38.1810 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z   61.3407 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z   62.8703 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z   64.0145 -148.765700    1.90  1.50      ml   
...                         ...       ...         ...     ...   ...     ...   
33889  2022-02-15T19:12:23.740Z   39.4215 -110.312333   -1.56  1.83      md   
33890  2022-02-15T19:04:27.070Z   38.8250 -122.797167    2.68  0.41      md   
33891  2022-02-15T19:00:38.627Z   38.5375 -119.382100    0.00  1.40      ml   
33892  2022-02-15T18:52:46.384Z   55.3724 -159.286100   15.00  1.80      ml   
33893  2022-02-15T18:48:53.076Z   61.8432 -147.521000   34.40  1.20      ml   

        nst     gap     dmin     rms  ...          

In [12]:
#replace the table in postgres with one without duplicates
model_df.to_sql('earthquake_raw', engine, if_exists='replace', index = False)

In [13]:
#replace the table in sqlite with one without duplicates
model_df2.to_sql('earthquake_raw', conn, if_exists='replace', index = False)

In [14]:
#copy data as csv as backup method
model_df.to_csv("earthquake_raw.csv",index=False)

In [15]:
# Close connection to SQLite database
conn.close()
connection.close()